In [1]:
import pandas as pd
import spacy
import random

In [2]:
#READ NORMALIZED DATA FOR TARGET VERSION OF PIPELINE
df = pd.read_csv("C:\\Users\\brush\\Projekt_NLP\\normalizedResult.tsv", sep="\t")

In [3]:
onlyPersonTweets=[]
onlyGroupTweets=[]
onlyPublicTweets=[]

personFile = open('targetPersonTweets.txt', 'r', encoding="utf-8")
temp = personFile.readlines()
for line in temp:
    onlyPersonTweets.append(line.replace(" \n", ""))
    
groupFile = open('targetGroupTweets.txt', 'r', encoding="utf-8")
temp = groupFile.readlines()
for line in temp:
    onlyGroupTweets.append(line.replace(" \n", ""))

publicFile = open('targetPublicTweets.txt', 'r', encoding="utf-8")
temp = publicFile.readlines()
for line in temp:
    onlyPublicTweets.append(line.replace(" \n", ""))

In [4]:
#CREATE TWO LISTS THAT CONTAIN EACH TEXT OF A TWEET AND ITS TARGET
df=df.dropna()
target=(df.iloc[0:8132 , 2 ].to_list())
text=df.iloc[0:8132, 1 ].to_list()
print(len(text))
print(len(target))

8081
8081


In [5]:
#CREATE DATAFRAME THAT CONTAINS TWEET TEXT AND TARGET
df = pd.DataFrame ({'Text': text , 'Target': target })

In [6]:
#SPLIT DATA INTO TRAIN AND TEST DATA
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    text, 
    target, 
    test_size=0.2, # 20% samples will go to test dataset
    random_state=2022, #-> same order of dataset
    stratify=target
)
print("Shape of X_train: ", len(X_train))
print("Shape of X_test: ", len(X_test))

Shape of X_train:  6464
Shape of X_test:  1617


In [7]:
###TARGET SPLITTING OF 0.8 DATA###
targetPerson=[]
targetGroup=[]
targetPublic=[]

counter=0
for t in X_train:
    if y_train[counter] == 0:
        targetPerson.append(t)
    elif y_train[counter] == 1:
        targetGroup.append(t)
    else:
        targetPublic.append(t)
    counter+=1
###TARGET SPLITTING END###

In [8]:
###UNIQUE WORD SPLITTING OF 0.8 DATA###
personWords=[]
groupWords=[]
publicWords=[]

for tweet in targetPerson:
    personWords.append(tweet.split())
personWords = sum(personWords, [])

for tweet in targetGroup:
    groupWords.append(tweet.split())
groupWords = sum(groupWords, [])

for tweet in targetPublic:
    publicWords.append(tweet.split())
publicWords = sum(publicWords, [])

personUnique=[]
groupUnique=[]
publicUnique=[]

for word in personWords:
    if word not in personUnique:
        personUnique.append(word)
        
for word in groupWords:
    if word not in groupUnique:
        groupUnique.append(word)
        
for word in publicWords:
    if word not in publicUnique:
        publicUnique.append(word)
        
onlyPerson=[]

for word in personUnique:
    if word not in groupUnique:
        if word not in publicUnique:
            onlyPerson.append(word)
            
onlyGroup=[]

for word in groupUnique:
    if word not in personUnique:
        if word not in publicUnique:
            onlyGroup.append(word)
            
onlyPublic=[]

for word in publicUnique:
    if word not in personUnique:
        if word not in groupUnique:
            onlyPublic.append(word)
###WORD SPLITTING END###

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
import xgboost as xgb

#1. create a pipeline object
clf = Pipeline([
     ('vectorizer_tfidf',TfidfVectorizer()),    
     #('KNN', KNeighborsClassifier())
     #('SVM', SVC()),
     #('rfc', RandomForestClassifier()) 
     ('xgb', xgb.XGBClassifier(objective="binary:logistic", random_state=42)) 
])

#2. fit with X_train and y_train
clf.fit(X_train, y_train)


#3. get the predictions for X_test and store it in y_pred
y_pred = clf.predict(X_test)

#4. print the classfication report BEFORE ANALYZER
classificationBefore = classification_report(y_test, y_pred)

#5. find accuracy scores
accuracyBefore = accuracy_score(y_test, y_pred)

personIst=0
groupIst=0
publicIst=0
for p in y_pred:
    if p == 0:
        personIst+=1
    if p == 1:
        groupIst+=1
    if p == 2:
        publicIst+=1
print("Person: " + str(personIst))
print("Group: " + str(groupIst))
print("Public: " + str(publicIst))

Person: 563
Group: 670
Public: 384


In [10]:
#6. run analyzer for target correction
y_pred2=y_pred
countList=[]
personCount=0
groupCount=0
publicCount=0
counter=0
single_pronouns=["du", "dich", "deinen"]
single_adress=["herr", "herrn", "frau", "junge", "nazi"]
single_politicians=["merkel", "söder", "spahn", "soeder", "laschet", "baerbock", "bundeskanzlerin", "bundeskanzler", "kanzler", "kanzlerin"]
group_pronouns=["euch", "eure", "euer", "deren", "ihr"]
group_politics=["spd", "cdu", "csu", "afd", "grünen", "grüne", "union", "linken", "nazis", "rechten", "partei", "land", "terroristen", "demokratie"]
group_countries=["deutschland", "deutsche", "bürger"]
public_pronouns=["alle", "wir", "menschen", "volk", "bevölkerung", "welt", "bewohner"]
testList=public_pronouns

for s in y_pred2:
    if y_pred2[counter] == 1:
        doc=X_test[counter].split()
        for token in doc:
            if token in onlyPerson:
                personCount+=1.2
            if token in single_pronouns or token in single_adress or token in single_politicians:
                personCount+=1.2
            if token in onlyGroup:
                groupCount+=0.9
            if token in group_pronouns or token in group_politics or token in group_countries:
                groupCount+=0.9
        if personCount > groupCount:
            y_pred2[counter]=0
        personCount=0
        groupCount=0
    counter+=1
personCount=0
groupCount=0
publicCount=0
counter=0

personIst2=0
groupIst2=0
publicIst2=0
for p in y_pred2:
    if p == 0:
        personIst2+=1
    if p == 1:
        groupIst2+=1
    if p == 2:
        publicIst2+=1

#7. print the classfication report AFTER ANALYZER
classificationAfter = classification_report(y_test, y_pred2)

#8. find accuracy scores
accuracyAfter = accuracy_score(y_test, y_pred2)
    
for s in y_pred2:
    if y_pred2[counter] == 0:
        doc=X_test[counter].split()
        for token in doc:
            if token in onlyPerson:
                personCount+=1
            if token in single_pronouns or token in single_adress or token in single_politicians:
                personCount+=1
            if token in onlyGroup:
                groupCount+=0.5
            if token in group_pronouns or token in group_politics or token in group_countries:
                groupCount+=0.5
        if groupCount > personCount:
            y_pred2[counter]=1
        personCount=0
        groupCount=0
    counter+=1
personCount=0
groupCount=0
publicCount=0
counter=0

personIst3=0
groupIst3=0
publicIst3=0
for p in y_pred2:
    if p == 0:
        personIst3+=1
    if p == 1:
        groupIst3+=1
    if p == 2:
        publicIst3+=1

#7. print the classfication report AFTER ANALYZER
classificationAfter2 = classification_report(y_test, y_pred2)

#8. find accuracy scores
accuracyAfter2 = accuracy_score(y_test, y_pred2)

for s in y_pred2:
    if y_pred2[counter] == 0 or y_pred2[counter] == 1:
        doc=X_test[counter].split()
        for token in doc:
            if token in onlyPerson:
                personCount+=1
            if token in single_pronouns or token in single_adress or token in single_politicians:
                personCount+=1
            if token in onlyGroup:
                groupCount+=1
            if token in group_pronouns or token in group_politics or token in group_countries:
                groupCount+=1
            if token in onlyPublic:
                publicCount+=1
            if token in public_pronouns:
                publicCount+=1
        if publicCount > personCount and publicCount > groupCount:
            y_pred2[counter]=2
        personCount=0
        groupCount=0
        publicCount=0
    counter+=1

personIst4=0
groupIst4=0
publicIst4=0
for p in y_pred2:
    if p == 0:
        personIst4+=1
    if p == 1:
        groupIst4+=1
    if p == 2:
        publicIst4+=1

#7. print the classfication report AFTER ANALYZER
classificationAfter3 = classification_report(y_test, y_pred2)

#8. find accuracy scores
accuracyAfter3 = accuracy_score(y_test, y_pred2)

###################################################################
# print(testList)
# freq=[]
# for i in testList:
#     freq.append(0)
# for p in onlyPersonTweets:
#     doc=onlyPersonTweets[counter].split()
#     for token in doc:
#         if token.lower() in testList:
#             c=0
#             for w in testList:
#                 if token.lower() == w:
#                     freq[c]+=1
#                 c+=1
#     counter+=1
# counter=0
# print(freq)
# freq.clear()

# for i in testList:
#     freq.append(0)
# for p in onlyGroupTweets:
#     doc=onlyGroupTweets[counter].split()
#     for token in doc:
#         if token.lower() in testList:
#             c=0
#             for w in testList:
#                 if token.lower() == w:
#                     freq[c]+=1
#                 c+=1
#     counter+=1
# counter=0
# print(freq)
# freq.clear() 

# for i in testList:
#     freq.append(0)
# for p in onlyPublicTweets:
#     doc=onlyPublicTweets[counter].split()
#     for token in doc:
#         if token.lower() in testList:
#             c=0
#             for w in testList:
#                 if token.lower() == w:
#                     freq[c]+=1
#                 c+=1
#     counter+=1
# counter=0
# print(freq)
# freq.clear() 

In [11]:
print("BEFORE ANALZYER:")
print(classificationBefore)
print("The accuracy of prediction is: ", accuracyBefore)
print("IST-WERT DAVOR:")
print("Person: " + str(personIst))
print("Group: " + str(groupIst))
print("Public: " + str(publicIst))
print("###############################################")
print("AFTER ANALYZER 1:")
print(classificationAfter)
print("The accuracy of prediction is: ", accuracyAfter)
print("IST-WERT nach Durchlauf 1:")
print("Person: " + str(personIst2))
print("Group: " + str(groupIst2))
print("Public: " + str(publicIst2))
print("AFTER ANALYZER 2:")
print(classificationAfter2)
print("The accuracy of prediction is: ", accuracyAfter2)
print("IST-WERT nach Durchlauf 2:")
print("Person: " + str(personIst3))
print("Group: " + str(groupIst3))
print("Public: " + str(publicIst3))
print("AFTER ANALYZER 3:")
print(classificationAfter3)
print("The accuracy of prediction is: ", accuracyAfter3)
print("IST-WERT nach Durchlauf 3:")
print("Person: " + str(personIst4))
print("Group: " + str(groupIst4))
print("Public: " + str(publicIst4))

BEFORE ANALZYER:
              precision    recall  f1-score   support

           0       0.67      0.66      0.67       571
           1       0.65      0.68      0.66       642
           2       0.51      0.49      0.50       404

    accuracy                           0.62      1617
   macro avg       0.61      0.61      0.61      1617
weighted avg       0.62      0.62      0.62      1617

The accuracy of prediction is:  0.6239950525664811
IST-WERT DAVOR:
Person: 563
Group: 670
Public: 384
###############################################
AFTER ANALYZER 1:
              precision    recall  f1-score   support

           0       0.58      0.75      0.65       571
           1       0.69      0.53      0.60       642
           2       0.51      0.49      0.50       404

    accuracy                           0.59      1617
   macro avg       0.59      0.59      0.58      1617
weighted avg       0.60      0.59      0.59      1617

The accuracy of prediction is:  0.5949288806431664
IS